# Import

In [1]:
import os
import nltk
import json
from time import gmtime, strftime
from nltk import sent_tokenize
from tqdm import tqdm
from itertools import chain

import multiprocessing
import parmap


from datasets import load_dataset, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer, AutoConfig, TrainingArguments, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer
from accelerate import notebook_launcher

import pickle

# Load Dataset

In [2]:
data_path = "../data"
original_train_datasets_path = data_path + "/original_datasets/TS1"
original_valid_datasets_path = data_path + "/original_datasets/VS1"

current_dir = os.getcwd()
current_time = strftime("%Y-%m-%d-%H:%M:%S", gmtime())
model_output_dir = './BERT'+"_"+current_time
model_cache_dir = "./BERT_cache"
raw_datasets_cache_name = ".raw_datasets_cache"
raw_datasets_cache_path = os.path.join(current_dir, raw_datasets_cache_name)

train_sentence_list_file_name = "train_sentence_list.txt"
valid_sentence_list_file_name = "valid_sentence_list.txt"
tokenizer_name = "tokenizer_aihub_news"
tokenized_datasets_folder_name = ["BERT_tokenized_datasets", "BERT_tokenized_datasets_1", "BERT_tokenized_datasets_2", "tokenized_datasets_3"]
grouped_tokenized_datasets_folder_name = "BERT_grouped_tokenized_datasets"

old_model_name = 'bert-base-uncased'

In [3]:
num_proc = multiprocessing.cpu_count()

In [9]:
# def listize_dataset(json_file_path):    
#     sentence_list = []

#     with open(json_file_path, 'r', encoding='UTF-8') as f:
#         json_object = json.load(f)

#     for line in json_object['SJML']['text']:
#         raw_text = line['content'].replace('..', '.')
#         raw_text_list = sent_tokenize(raw_text)
        
#         for item in raw_text_list:
#             if len(item) < 5:
#                 continue
#             else:
#                 sentence_list.append(item.strip())

#     return sentence_list

In [10]:
# file_path_list = []

# file_list = os.listdir(os.path.join(original_train_datasets_path))
# for dir_name in file_list:
#     file_path = os.path.join(original_train_datasets_path, dir_name)

#     json_file_list = os.listdir(file_path)
#     for json_file_name in json_file_list:
#         json_file_path = os.path.join(file_path, json_file_name)

#         file_path_list.append(json_file_path)

# train_sentence_list_of_list = parmap.map(listize_dataset, file_path_list, pm_pbar=True, pm_processes=num_proc)

NameError: name 'parmap' is not defined

In [6]:
# file_path_list = []
# file_list = os.listdir(os.path.join(original_valid_datasets_path))
# for dir_name in file_list:
#     file_path = os.path.join(original_valid_datasets_path, dir_name)

#     json_file_list = os.listdir(file_path)
#     for json_file_name in json_file_list:
#         json_file_path = os.path.join(file_path, json_file_name)

#         file_path_list.append(json_file_path)

# valid_sentence_list_of_list = parmap.map(listize_dataset, file_path_list, pm_pbar=True, pm_processes=num_proc)

In [7]:
# train_sentence_list = list(chain(*train_sentence_list_of_list))
# valid_sentence_list = list(chain(*valid_sentence_list_of_list))

In [8]:
# with open(os.path.join(data_path, train_sentence_list_file_name), 'w') as fw:
#     for sentence in train_sentence_list:
#         fw.write(sentence + "\n")

# with open(os.path.join(data_path, valid_sentence_list_file_name), 'w') as fw:
#     for sentence in valid_sentence_list:
#         fw.write(sentence + "\n")

In [11]:
raw_datasets = load_dataset('text', data_files={"train": os.path.join(data_path, train_sentence_list_file_name), "valid": os.path.join(data_path, valid_sentence_list_file_name)}, cache_dir=raw_datasets_cache_path)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /home/work/sekim_backup/KoLM_pretrain/KoBERT/.raw_datasets_cache/text/default-1a9a91f22c227c27/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 53742969
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 6268737
    })
})

# Tokenizing

### from scratch

In [15]:
# # training a tokenizer from scratch
# def get_training_corpus(batch_size=10000):
#     for dataset in [raw_datasets['train'], raw_datasets['valid']]:
#         for start_idx in range(0, len(dataset), batch_size):
#             yield dataset[start_idx : start_idx + batch_size]["text"]
        
# old_tokenizer = AutoTokenizer.from_pretrained(old_model_name)
# tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus(), 15000)
# tokenizer.save_pretrained(tokenizer_name)

('tokenizer_aihub_news/tokenizer_config.json',
 'tokenizer_aihub_news/special_tokens_map.json',
 'tokenizer_aihub_news/vocab.txt',
 'tokenizer_aihub_news/added_tokens.json',
 'tokenizer_aihub_news/tokenizer.json')

In [13]:
# raw_datasets_1 = raw_datasets.select(range(20000000))
# raw_datasets_2 = raw_datasets.select(range(20000000, 40000000))
# raw_datasets_3 = raw_datasets.select(range(40000000, len(raw_datasets)))

In [7]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [17]:
# def preprocess_texts(examples):
   #  tokenized_inputs = tokenizer(
   #     examples["text"], return_special_tokens_mask=True, truncation=True, max_length=tokenizer.model_max_length
   #  )
   #  return tokenized_inputs

In [18]:
# tokenized_datasets = raw_datasets.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)

Map (num_proc=48):   0%|          | 0/53742969 [00:00<?, ? examples/s]

Map (num_proc=48):   0%|          | 0/6268737 [00:00<?, ? examples/s]

In [17]:
# tokenized_datasets_1 = raw_datasets_1.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
# tokenized_datasets_1.save_to_disk(os.path.join(current_dir, tokenized_datasets_folder_name[1]))

In [18]:
# tokenized_datasets_2 = raw_datasets_2.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
# tokenized_datasets_2.save_to_disk(os.path.join(current_dir, tokenized_datasets_folder_name[2]))

In [19]:
# tokenized_datasets_3 = raw_datasets_3.map(preprocess_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
# tokenized_datasets_3.save_to_disk(os.path.join(current_dir, tokenized_datasets_folder_name[3]))

In [20]:
# tokenized_datasets_1 = load_from_disk(os.path.join(current_dir, tokenized_datasets_folder_name[1]))

In [21]:
# tokenized_datasets_2 = load_from_disk(os.path.join(current_dir, tokenized_datasets_folder_name[2]))

In [22]:
# tokenized_datasets_3 = load_from_disk(os.path.join(current_dir, tokenized_datasets_folder_name[3]))

In [23]:
# tokenized_datasets = concatenate_datasets([tokenized_datasets_1, tokenized_datasets_2, tokenized_datasets_1])

In [19]:
# tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 53742969
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 6268737
    })
})

In [20]:
# tokenized_datasets.save_to_disk(os.path.join(current_dir, tokenized_datasets_folder_name[0]))

Saving the dataset (0/33 shards):   0%|          | 0/53742969 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/6268737 [00:00<?, ? examples/s]

In [26]:
# tokenized_datasets = load_from_disk(os.path.join(current_dir, tokenized_datasets_folder_name[0]))

# Grouping

In [22]:
# # Main data processing function that will concatenate all texts from our dataset and generate chunks of
# # max_seq_length.
# def group_texts(examples):
#     # Concatenate all texts.
#     concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#     # customize this part to your needs.
#     if total_length >= tokenizer.model_max_length:
#         total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
#     # Split by chunks of max_len.
#     result = {
#         k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
#         for k, t in concatenated_examples.items()
#     }
#     return result

In [23]:
# tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=16)
# # shuffle dataset
# tokenized_datasets = tokenized_datasets.shuffle(seed=34)

# print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")

Map (num_proc=16):   0%|          | 0/53742969 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/6268737 [00:00<?, ? examples/s]

the dataset contains in total 1024 tokens


In [29]:
# tokenized_datasets

In [24]:
# tokenized_datasets.save_to_disk(os.path.join(current_dir, grouped_tokenized_datasets_folder_name))

Saving the dataset (0/31 shards):   0%|          | 0/4215240 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/532142 [00:00<?, ? examples/s]

In [4]:
tokenized_datasets = load_from_disk(os.path.join(current_dir, grouped_tokenized_datasets_folder_name))

# DDP Train

In [5]:
model_config = AutoConfig.from_pretrained(old_model_name)

In [8]:
# DDP Train
def train_trainer_ddp():
    model = AutoModelForMaskedLM.from_pretrained(old_model_name, config=model_config, cache_dir=model_cache_dir)
    model.resize_token_embeddings(len(tokenizer))

    training_args = TrainingArguments(
        output_dir = model_output_dir,
        logging_dir="runs/"+model_output_dir,
        do_train = True,
        do_eval = True,
        per_device_train_batch_size = 48,
        per_device_eval_batch_size = 48,        
        evaluation_strategy = "steps",
        eval_steps=1,
        save_strategy="steps",
        save_steps=5000,
        logging_steps = 100,
        learning_rate = 5e-5,
        weight_decay = 0,
        adam_epsilon = 1e-8,
        max_grad_norm = 1.0,
        num_train_epochs = 2,
        disable_tqdm="false",
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm=True, 
                                                mlm_probability=0.15,)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['valid'].select(range(100000))
    )   

    trainer.train()

notebook_launcher(train_trainer_ddp, args=(), num_processes=4)

Launching training on 4 GPUs.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model

{'eval_loss': 10.620137214660645, 'eval_runtime': 74.1021, 'eval_samples_per_second': 1349.489, 'eval_steps_per_second': 7.031, 'epoch': 0.0}


KeyboardInterrupt: 

: 

# Prepare Training

In [34]:
# model_config = AutoConfig.from_pretrained('bert-base-uncased')

In [35]:
# model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased', config=model_config, cache_dir=model_cache_dir)
# model.resize_token_embeddings(len(tokenizer))

In [36]:
# training_args = TrainingArguments(
#     output_dir = model_output_dir,
#     overwrite_output_dir = True,
#     do_train = True,
#     do_eval = True,
#     per_device_train_batch_size = 32,
#     per_device_eval_batch_size = 32,
#     logging_steps = 50,
#     prediction_loss_only = True,
#     learning_rate = 5e-5,
#     weight_decay = 0,
#     adam_epsilon = 1e-8,
#     max_grad_norm = 1.0,
#     num_train_epochs = 2,
#     save_steps = -1
# )

In [37]:
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
#                                                 mlm=True, 
#                                                 mlm_probability=0.15,)

In [38]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=tokenized_datasets,
# )

In [39]:
# trainer.train()